<a href="https://colab.research.google.com/github/FatimaJahara/Teacher-Student-Knowledge-Distillation/blob/main/text_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Sentiment Analysis with BERT**

* **Teacher Model:**  A pre-trained `bert-base-uncased` model fine-tuned for binary sentiment
  classification.
* **Student Model:** A smaller BERT model with reduced hidden size, fewer layers, and fewer
  attention heads, initialized with a configuration derived from the teacher's.
* **Distillation:**  Similar to the MNIST example, the student is trained with a combination of
  cross-entropy loss and KL divergence-based distillation loss, using softened probabilities
  from both the teacher and student.
* **Dataset:** A small, dummy dataset is included for demonstration purposes.  In a real-world
  scenario, you would replace this with a larger, more representative dataset (e.g., IMDB, SST-2).
* **Hugging Face Transformers:** This part utilizes the Hugging Face `transformers` library for
  easy access to pre-trained BERT models and tokenizers, as well as utilities for training.

**Key Concepts Illustrated:**

*   **Knowledge Distillation:**  The core concept of transferring knowledge from a teacher to a
    student network.
*   **Soft Targets:**  Using the teacher's output probabilities (softened by a temperature
    parameter) as targets for the student.
*   **Temperature Scaling:**  The use of a temperature parameter to control the "softness" of the
    probability distributions.
*   **KL Divergence:**  Measuring the difference between the teacher's and student's softened
    probability distributions.
*   **Combined Loss:**  Combining the standard cross-entropy loss with the distillation loss.
*   **Model Compression:**  The student model is significantly smaller than the teacher model,
    demonstrating potential for model compression.
*   **PyTorch:**  Implementation using the PyTorch deep learning framework.
*   **Hugging Face Transformers:** Using the transformers library for BERT models.
* **DataLoader:** Using PyTorch's DataLoader to efficiently load and batch data.

**Structure of the Notebook:**

The code is organized into the following sections for both examples:

1.  **Model Definition(s):** Defines the `TeacherNet` and `StudentNet` classes (MNIST) and functions
    to load/create the teacher and student BERT models (Sentiment Analysis).
2.  **Data Loading:**  Includes functions to load and pre-process the MNIST dataset and a class and
    function to prepare the sentiment analysis dataset and DataLoaders.
3.  **Training Function(s):**  Contains `train_teacher`, `train_student_with_distillation`, `test`,
 and `train_student`.
4.  **Main Execution:**  The `main` function orchestrates the entire process: loading data,
    training the teacher, training the student with and without distillation, and reporting results.

**How to Run:**

1.  **Install Dependencies:**
    ```bash
    pip install torch torchvision transformers tqdm
    ```
2.  **Run the Notebook:** Execute the code cells sequentially.  The MNIST dataset will be
    downloaded automatically.  The BERT model will also be downloaded.

**Expected Output:**

The notebook will print training progress (loss values) and final accuracy scores for the
teacher model, the student model trained with distillation, and the student model trained without
distillation.  You should observe that the distilled student performs better than the baseline
student, demonstrating the effectiveness of knowledge distillation. For the BERT example, training
and validation metrics will be printed.

In [7]:
# from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW, get_linear_schedule_with_warmup

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import torch
from torch.optim import AdamW
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm  # For progress bars

In [6]:
# --- 1. Data Preparation  ---

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Ensure text is a string
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


def create_data_loaders(train_texts, train_labels, val_texts, val_labels, tokenizer, batch_size=16, max_length=128):

    train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, max_length)
    val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, max_length)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, val_dataloader

In [8]:
# --- 2. Model Definitions ---

def load_teacher_model(teacher_model_name="bert-base-uncased"):
    teacher_model = BertForSequenceClassification.from_pretrained(teacher_model_name, num_labels=2)  # Binary classification
    tokenizer = BertTokenizer.from_pretrained(teacher_model_name)
    return teacher_model, tokenizer


def create_student_model(teacher_config, num_labels=2):
    student_config = BertConfig(
        vocab_size=teacher_config.vocab_size,
        hidden_size=teacher_config.hidden_size // 2,  # Reduce hidden size
        num_hidden_layers=teacher_config.num_hidden_layers // 2,  # Reduce layers
        num_attention_heads=teacher_config.num_attention_heads // 2,  # Reduce heads
        intermediate_size=teacher_config.intermediate_size // 2,  # Reduce intermediate size
        max_position_embeddings=teacher_config.max_position_embeddings,
        num_labels=num_labels,
    )
    student_model = BertForSequenceClassification(student_config)
    return student_model

In [9]:

# --- 3. Distillation Training Function ---

def train_student(teacher_model, student_model, train_dataloader, val_dataloader, optimizer, scheduler, epochs=3, temperature=2.0, alpha=0.5, device='cpu'):
    teacher_model.to(device)
    student_model.to(device)
    teacher_model.eval()  # Teacher is always in eval mode

    for epoch in range(epochs):
        student_model.train()
        total_loss = 0
        total_distill_loss = 0
        total_ce_loss = 0

        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            # Teacher logits
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids, attention_mask=attention_mask).logits

            # Student logits
            student_logits = student_model(input_ids, attention_mask=attention_mask).logits

            # --- Distillation Loss ---
            soft_targets = F.softmax(teacher_logits / temperature, dim=-1)
            soft_student_logits = F.log_softmax(student_logits / temperature, dim=-1)  # LogSoftmax for KLDiv
            distillation_loss = F.kl_div(soft_student_logits, soft_targets, reduction='batchmean') * (temperature ** 2)  # Scale by T^2

            # --- Student Loss (Cross-Entropy) ---
            ce_loss = F.cross_entropy(student_logits, labels)

            # --- Combined Loss ---
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            total_distill_loss += distillation_loss.item()
            total_ce_loss += ce_loss.item()

        avg_train_loss = total_loss / len(train_dataloader)
        avg_distill_loss = total_distill_loss / len(train_dataloader)
        avg_ce_loss = total_ce_loss / len(train_dataloader)

        print(f"Epoch {epoch+1} - Avg Train Loss: {avg_train_loss:.4f}, Distill Loss: {avg_distill_loss:.4f}, CE Loss: {avg_ce_loss:.4f}")

        # --- Validation ---
        student_model.eval()
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = student_model(input_ids, attention_mask=attention_mask)
                _, predicted = torch.max(outputs.logits, dim=1)

                correct_predictions += (predicted == labels).sum().item()
                total_predictions += labels.size(0)

        accuracy = correct_predictions / total_predictions
        print(f"Epoch {epoch+1} - Validation Accuracy: {accuracy:.4f}")

In [10]:
# --- 4. Main Execution ---

def main():
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 1. Load Teacher Model and Tokenizer
    teacher_model, tokenizer = load_teacher_model()
    teacher_config = teacher_model.config

    # 2. Create Student Model
    student_model = create_student_model(teacher_config)

    # 3. Prepare Data (Example - Replace with your actual data)
    # Dummy data for demonstration (small and balanced)
    train_texts = [
        "This is a great movie!", "I really enjoyed this.", "Absolutely fantastic!",
        "This is a terrible movie.", "I hated it.", "Absolutely awful!"
    ]
    train_labels = [1, 1, 1, 0, 0, 0]  # 1: Positive, 0: Negative

    val_texts = [
        "It was good.", "Not bad at all.",
        "It was very bad.", "I did not like it."
    ]
    val_labels = [1, 1, 0, 0]

    train_dataloader, val_dataloader = create_data_loaders(train_texts, train_labels, val_texts, val_labels, tokenizer)

    # 4. Optimizer and Scheduler
    optimizer = AdamW(student_model.parameters(), lr=5e-5, eps=1e-8)  # Use AdamW
    epochs = 3
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # 5. Train Student with Distillation
    train_student(teacher_model, student_model, train_dataloader, val_dataloader, optimizer, scheduler, epochs=epochs, device=device)



if __name__ == "__main__":
    main()

Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1 - Avg Train Loss: 0.3788, Distill Loss: 0.0864, CE Loss: 0.6711
Epoch 1 - Validation Accuracy: 0.5000


Epoch 2/3: 100%|██████████| 1/1 [00:03<00:00,  3.71s/it]


Epoch 2 - Avg Train Loss: 0.3771, Distill Loss: 0.0120, CE Loss: 0.7423
Epoch 2 - Validation Accuracy: 0.5000


Epoch 3/3: 100%|██████████| 1/1 [00:04<00:00,  4.78s/it]


Epoch 3 - Avg Train Loss: 0.3668, Distill Loss: 0.0077, CE Loss: 0.7258
Epoch 3 - Validation Accuracy: 0.5000
